<i style='color:green'>Cleaning up the Quora data</i>
<hr>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import re
from IPython.core.display import display, HTML

print 'ready'

ready


In [3]:
#############################################################################################
## First pass
## The data originally comes in a file called "questions.csv" which has a few small
## annoyances. 
##
## One is that not all quotation marks use the same encoding, which I've 
## cleaned up in VI. Another is that there were a number of lines that had 
## unintentional newlines in them, resulting in misread records, which I've
## also addressed by hand.  
##
## Finally, both the question1 and question2 fields have quotation marks around 
## them which are often in addition to quotation marks that are actually part of the text.
## This is a standard CSV encoding strategy that allows commas to be used as delimiters.
## However, it results in many instances of multiple quoting (quotation marks around
## quotation marks, playing different roles) that can run as many as 3 deep. It's standard,
## and works, but I find it messy.
##
## I decided to use a TAB as a field delimiter for this data instead.  Since all the
## questions come from online forms, there are definitely no tabs in the text itself.
## I also replaced all instances of multiple quoting in the original file with single 
## quotes, since quotation marks are now just part of the text and play no role in CSV 
## processing.
## 
## The perl code to do this is:
##
## ------------------------------------------------------------------------------------------
## print "id\tqid1\tqid2\tquestion1\tquestion2\tis_duplicate\n";
## open (Q, "questions.csv");
## while(<Q>)
## {
##    next if ($_ =~ /is_duplicate/); # skip the header line
##    chop;
##    my($id,$qid1,$qid2,$q1,$q2,$y)=($_=~ m/(\d+),(\d+),(\d+),\"(.*?)\",\"(.*?)\",\"(\d)\"$/);
##    $q1 =~ s/\"{1,5}/\"/g;
##    $q2 =~ s/\"{1,5}/\"/g;
##    print "$id\t$qid1\t$qid2\t$q1\t$q2\t$y\n";
## }  
## close Q;
## ------------------------------------------------------------------------------------------
##
## which can be run as: ./to_tabby.pl > tabby_questions.csv
##
#############################################################################################

# quoting=3 makes pandas treat quote characters as normal text
data = pd.read_csv('DATA/tabby_questions.csv', sep='\t', quoting=3)

dropqueue = []
print 'data ready'

data ready


In [5]:
print len(data)
data.head(3)

404351


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0


In [38]:
#############################################################################################
##  Drop any rows where either question1 or question2 is missing
## .. just a couple of these
#############################################################################################

indices = data.question2[data.question1.isnull()].index; 
data.drop(indices, axis=0, inplace=True);

indices = data.question2[data.question2.isnull()].index; 
data.drop(indices, axis=0, inplace=True);


# drop the 'id' column, since it's identical to the index
dropqueue.append('id')

# rename 'is_duplicate' to 'Y'
data.rename(columns={'is_duplicate':'Y'}, inplace=True)

print '.. updated'

.. updated


In [ ]:
#############################################################################################
##  Drop any rows that have nothing left after any stop words are removed
##  .. using spark to speed this up a bit
#############################################################################################

In [22]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext()

In [46]:
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)

dd = data.copy()
dd.reset_index(inplace=True)
dd.rename(columns={'index':'linekey'}, inplace=True)
dd = dd[['linekey', 'question1', 'question2']]

sparkdf = sqlsc.createDataFrame(dd)
rdd = sparkdf.rdd

In [52]:
rdd.take(2)

[Row(linekey=0, question1=u'What is the step by step guide to invest in share market in india?', question2=u'What is the step by step guide to invest in share market?'),
 Row(linekey=1, question1=u'What is the story of Kohinoor (Koh-i-Noor) Diamond?', question2=u'What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?')]

In [99]:
# this will produce two lists of counts (one for each question column)
# of words not including stopwords.  we'll use it to pull the indices of 
# questions that are too short (fewer than 2 words when stop words are taken out)

import nltk
from nltk.tokenize import RegexpTokenizer
stopwords = nltk.corpus.stopwords.words()
tokenizer = RegexpTokenizer(r'\w+')

q1counts=rdd.map(          \
      lambda line:(        \
            line.linekey,  \
            len(set([x for x in tokenizer.tokenize(line.question1) if x not in stopwords]))))

print 'q1counts ready'

q2counts=rdd.map(          \
      lambda line:(        \
            line.linekey,  \
            len(set([x for x in tokenizer.tokenize(line.question2) if x not in stopwords]))))

print 'q2counts ready'

q1df = q1counts.toDF().toPandas()
print 'q1df ready'

q2df = q2counts.toDF().toPandas()
print 'q2df ready'

q1counts ready
q2counts ready
q1df ready
q2df ready


In [137]:
# finally, remove the offending rows
removelist = (q1df[q1df['_2'] <= 2].index).tolist()
removelist.extend((q2df[q2df['_2'] <= 2].index).tolist())

print 'data has', len(data), 'rows before removal'

data.drop(removelist, axis=0, inplace=True);
print 'data has', len(data), 'rows after removal'

data has 404349  rows before removal
data has 396628  rows after removal


In [381]:
sc.stop()

In [382]:
#############################################################################################
##  re-encoding question IDs and duplicates
#############################################################################################

<i style='color:green'>
'qid1' and 'qid2' are not individual question IDs.  They're more like question keys where a certain ID will appear as many times as that specific (exact duplicate) question appears. 
<p>
Unfortunately, in many of these cases, the first pair with the given qid is unreleated to the others (ie - the matches aren't correct), which means that the keys are unusable. Also, there are no individual question IDs. Both issues will be remedied below.
<p>Finally, some researchers have noticed that the IDs imply a chronology.  That is, higher IDs belong to questions that were submitted later in time at the quora web site.  However, that also means that those later questions were subject to some degree of filtering during the input process, which means that there are likely to be more questions that don't match any other questions in the higher ID registers than in the lower ones.  Removing that information is another good reason reset the IDs.
<hr>
The example below is typical of cases where the IDs are incorrect. Note that the ID, 77116, in most cases refers to a question about iPhones, but in one case refers to a question about color.</i>

In [145]:
ql = data.qid1.value_counts()
ql.append(data.qid2.value_counts())

dropqueue.append('qid2')  # these columns will be replaced below, so we don't need them
dropqueue.append('qid1')

pd.options.display.max_colwidth = 150
sample = 0; sample = 50;

data[['qid1', 'qid2', 'question1', 'question2', 'Y']][data.qid1 == ql.index[sample]]

,qid1,qid2,question1,question2,Y
38709,77116,77117,What is the opposite color green?,What color is the opposite of green?,1
52618,77116,104709,Where can I get statistics on iPhone application downloads?,IPhone application: is there any QA app?,0
59062,77116,117489,Where can I get statistics on iPhone application downloads?,"How can I download ""unlock iPhone tool""?",0
286341,77116,562725,Where can I get statistics on iPhone application downloads?,Where can I get download statistics on every iPhone and Android application?,0
332491,77116,651977,Where can I get statistics on iPhone application downloads?,How much would it cost to develop an app similar to the Yo app?,0


In [146]:
#############################################################################################
##  dropping columns in dropqueue
#############################################################################################
data.drop(dropqueue, axis=1, inplace=True)
data = data.reset_index(drop=True)

#############################################################################################
##  end housekeeping
#############################################################################################
data.head(2)

,question1,question2,Y
0,What is the step by step guide to invest in share market in india?,What is the step by step guide to invest in share market?,0
1,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?,0


<i style='color:green'>
In the steps below, questions that are exact duplicates will be identified and assigned group numbers in columns 'g1', and 'g2', corresponding to the 'question1' and 'question2' columns. 
</i>

In [187]:
#############################################################################################
## figuring out which questions are exact duplicates 
#############################################################################################

# Pull all of the questions in the 'question1' column into a dataframe
d1 = data.copy()
d1 = d1['question1'].to_frame()
d1.reset_index(inplace=True)
d1.rename(columns={'index':'i', 'question1':'q'}, inplace=True)
print 'step: d1'

# Do the same with the questions in the 'question2' column
d2 = data.copy()
d2 = d2['question2'].to_frame()
d2.reset_index(inplace=True)

# At this point in the creation of d2, there's a column called 'index' that's identical to 
# the index. The lines below add a constant to put all the values in that column
# out of range of the original index (which was not done for d1) so that we can keep track 
# of which column the given question came from when they're all concatenated to one long list.
# In the end, just as in d1 above, d2 will have two columns, [i, q], where each
# i is a unique ID for the given q
d2['ii'] = d2['index'] + 1000000 
d2.drop('index', axis=1, inplace=True)
d2.rename(columns={'ii':'i', 'question2':'q'}, inplace=True)
print 'step: d2'

# make one large dataframe where each question has a unique ID
dd = pd.concat([d1, d2])
dd.reset_index(drop=True, inplace=True)

# group that large dataframe by question text
dfgroups = dd.groupby('q')

print 'step: number of groups in dfgroups', dfgroups.ngroups

# the group keys are the actual questions: 
# e.g. - ki = dfgroups.get_group('How much was reddit sold for?').index

# dfgroups keys hold unique questions, some but not all of which have 
# exact duplicates elsewhere in the data.  thus, in the two nested lists
# below (lk1 and lk2), many of the rows will have only one item while a
# smaller number will have more than one

lk1 = []  # group list for 'question1' column
lk2 = []  # group list for 'question2' column
for k in dfgroups.groups.keys():
    ki = dfgroups.get_group(k).i.tolist()
    ki_q1 = [x for x in ki if x < 1000000]
    ki_q2 = [(x - 1000000) for x in ki if x >=  1000000] # gets us back to real index values
    lk1.append(ki_q1)  # group number is not stored because it = the index value of this list
    lk2.append(ki_q2)
        
print 'step: lists made'


# finally, the actual assignments.  There are now two lists (lk1 and lk2), each
# containing nested lists of identical questions
assignations = pd.DataFrame(index=data.index)
assignations['g1'] = np.nan
assignations['g2'] = np.nan

# after transposing, each column heading represents a group ID
lk1_df = pd.DataFrame(lk1).T 
for groupnum in lk1_df.columns:
    z = lk1_df[groupnum].dropna()
    for item in z:
        # 'item' is the row number in 'data' in which this question is found
        # so this says that, "in row[item], question1 has the group id [groupnum]".
        assignations.set_value(item, 'g1', groupnum)
         
lk2_df = pd.DataFrame(lk2).T
for groupnum in lk2_df.columns:
    z = lk2_df[groupnum].dropna()
    for item in z:
        assignations.set_value(item, 'g2', groupnum)
            
            
data['g1'] = assignations['g1']
data['g2'] = assignations['g2']

print 'group numbers assigned'

step: d1
step: d2
step: number of groups in dfgroups 529136
step: lists made
group numbers assigned


In [ ]:
#############################################################################################
##  numbering individual questions properly
#############################################################################################

<i style='color:green'>
The questions also don't have individual IDs (as there are two questions per row, index IDs refer to pairs, not questions).  That might be useful, so this step adds them.
</i>

In [188]:
# now let's number each question
data['iid1'] = np.nan
data['iid2'] = np.nan
c = 0;
for r in data.index:
    c = c + 1;
    data.set_value(r, 'iid1', c)
    c = c + 1;
    data.set_value(r, 'iid2', c)
print 'done'
    

done


In [ ]:
#############################################################################################
##  creating metagroup tags
#############################################################################################

<i style='color:green'>
Finally, there are metagroups.  
<p>
A given question may, for example, have 5 exact duplicates, each of which is deemed equivalent in meaning to some other question that is not an exact duplicate.  Each of those questions, in turn, may have exact duplicates of their own, and then matching questions of their own, etc.
<p>
Altogether, then, there will be large groupings of questions that mean the same thing, which I'm calling metagroups (because I chose to use the term 'group' for exact duplicates).  Metagroups are identified below.
<p>
Although they won't be used in the final evalutation of sentences, they're useful as a way to break the data down for investigation.
</i>

In [222]:
# get a group list
G = data.g1.tolist()
G.extend(data.g2.tolist())
Gb = pd.Series(G).value_counts()
Gbi = Gb.index 

In [2]:
#############################################################################################
## identifying metagroups
#############################################################################################

metagroups = pd.DataFrame(index=range(0), columns=['metagroup'])
metagroups.reset_index(drop=True, inplace=True)

datagroups = data.copy()
datagroups = datagroups[['g1', 'g2' ,'Y']]  # to reduce the memory footprint of this section
datagroups['metagroup1'] = np.nan
datagroups['metagroup2'] = np.nan

def add_groups(r, GIDS):
    for ind in r.index:
        GIDS.add(r.loc[ind].g2)      # collect all groups indicated by matching questions
        GIDS.add(r.loc[ind].g1)       
    return GIDS

# Get all the questions with a given group ID in g1 and g2 (ie, exact duplicates)
# (GG, from above, is a list of group IDs for questions with at least one duplicate)
gc        = 0
trip      = 50000
metaindex = 0

for gi in Gbi: # Gbi is a list of all groups
    gc = gc + 1
    GIDS = set()                # set for group IDs for matching questions
    GIDS.add(gi)                # adding the first one .. more to come
    if (gc >= trip): print gc, '| gi = ', gi; trip = trip + 50000;
    
    rr=datagroups[((datagroups.g1.isin(GIDS))|(datagroups.g2.isin(GIDS)))&(datagroups.Y == 1)] # initial metagroup
    GIDS  = add_groups(rr, GIDS)
    
    c       = 0
    lastlen = len(GIDS)
    stop    = 0
    while((c < 10) & (stop == 0)):  # 10 seems to be more than enough
        # now GIDS is bigger, so there's a bigger metagroup too
        rr = datagroups[((datagroups.g1.isin(GIDS)) | (datagroups.g2.isin(GIDS))) & (datagroups.Y == 1)]
        GIDS = add_groups(rr, GIDS)
        
        c = c + 1
        newlen = len(GIDS)
        
        # eventually, GIDS stops getting bigger
        if (newlen == lastlen): stop = 1; 
        lastlen = newlen

    if len(GIDS) > 0:
        datagroups.set_value(datagroups[datagroups.g1.isin(GIDS)].index, 'metagroup1', metaindex)
        datagroups.set_value(datagroups[datagroups.g2.isin(GIDS)].index, 'metagroup2', metaindex)
        metaindex = metaindex + 1

print 'done'

done


In [226]:
data['metagroup1'] = datagroups['metagroup1']
data['metagroup2'] = datagroups['metagroup2']

In [227]:
data['linekey'] = data.index
data.head()

,question1,question2,Y,g1,g2,iid1,iid2,metagroup1,metagroup2,linekey
0,What is the step by step guide to invest in share market in india?,What is the step by step guide to invest in share market?,0,263854.0,335639.0,1.0,2.0,181604.0,520073.0,0
1,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?,0,103260.0,184453.0,3.0,4.0,22526.0,87244.0,1
2,How can I increase the speed of my internet connection while using a VPN?,How can Internet speed be increased by hacking through DNS?,0,228552.0,494549.0,5.0,6.0,123054.0,189746.0,2
3,Why am I mentally very lonely? How can I solve it?,"Find the remainder when [math]23^{24}[/math] is divided by 24,23?",0,106677.0,490131.0,7.0,8.0,381703.0,157523.0,3
4,"Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?",Which fish would survive in salt water?,0,412129.0,501558.0,9.0,10.0,26441.0,145424.0,4


In [350]:
#############################################################################################
## One final bit of cleanup - removing rows that have non-english characters in
## the questions.  Although that's a situation that will come up in the wild, for
## the purposes of this exploration it's just extra work.
#############################################################################################

import string

def isEnglish(s):
    return s.translate(None, string.punctuation).isalnum()

non_english = []
for i in data.index:
    q1 = data.loc[i].question1; q1 = q1.replace(" ", "");
    q2 = data.loc[i].question2; q2 = q2.replace(" ", "");
    
    if(isEnglish(q1)): donothing = 1
    else:              non_english.append(i)

    if(isEnglish(q2)): donothing = 1
    else:              non_english.append(i)
    
print 'done'

done


In [377]:
print len(data)
data.drop(non_english, axis=0, inplace=True);
print len(data)

388060
388060


In [384]:
#############################################################################################
## Saving the data
#############################################################################################

In [364]:
#############################################################################################
## put it in a database
#############################################################################################

import MySQLdb
from lib import trlib as trsubs

def dosql(sql):
    try:
        cursor.execute(sql)
        db.commit()
    except TypeError as e:
        print "could not update"
        print (e)
 
db, cursor = trsubs.openmysql()

In [372]:
# linekey, question1, question2, Y, g1, g2, iid1, iid2, metagroup1, metagroup2
# the longest question is 910 characters
sql = "drop table quora_dd"
dosql(sql)

sql = "create table if not exists quora_dd (linekey int, question1 varchar(2000), question2 varchar(2000), Y int, g1 int, g2 int, iid1 int, iid2 int, metagroup1 int, metagroup2 int)"
dosql(sql)

In [380]:
# fill the table with the current data
sql = "insert into quora_dd (linekey, question1, question2, Y, g1, g2, iid1, iid2, \
       metagroup1, metagroup2) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
print sql, '\n'

count     = 0
truecount = 0
mark      = 20000

locald = data.copy()
for i in locald.index:
    lk = str(locald.loc[i]['linekey'])
    q1 = str(locald.loc[i].question1)
    q2 = str(locald.loc[i].question2)

    var1  = lk
    var2  = q1
    var3  = q2
    var4  = str(locald.loc[i]['Y'])
    var5  = str(locald.loc[i]['g1'])
    var6  = str(locald.loc[i]['g2'])
    var7  = str(locald.loc[i]['iid1'])
    var8  = str(locald.loc[i]['iid2'])
    var9  = str(locald.loc[i]['metagroup1'])
    var10 = str(locald.loc[i]['metagroup2'])

    cursor.execute(sql, (var1, var2, var3, var4, var5, var6, var7, var8, var9, var10))
    
    if(count >= mark): 
        print truecount; 
        print q1; 
        print q2, '\n'; 
        count = 0;
    count     = count + 1
    truecount = truecount + 1

db.commit()
del locald
print 'done'

insert into quora_dd (linekey, question1, question2, Y, g1, g2, iid1, iid2,        metagroup1, metagroup2) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s) 

20000
What do the most widely used Japanese textbooks teach about Japan's invasion of China?
What do the Japanese think about China? 

40000
How are the menus made internal in CSGO?
How are cheat menu's made internal in CSGO? 

60000
Can ions exist in nature as molecule?
How do ions exist in nature? 

80000
What are the strengths and weaknesses of the advising system at Indiana Wesleyan University?
What are the strengths and weaknesses of the advising system at Ohio Wesleyan University? 

100000
Did the students strikes in Montreal have any real profound effect on the collective consciousness of Quebec?
Have you ever taken any sort of advice from someone that you regretted later? Did it have any profound effect on your life at all? 

120000
How can I lose an extreme amount of weight?
How can I lose post marriage weight? 

140000
Wh

In [385]:
trsubs.closemysql(db)